# Build a test data set of house price data

In this tutorial, we'll build a test data set. The dataset will be a simple real estate dataset showing attributes such as price, number of bedrooms, parking etc. The data is entirely fabricated. Once created, we'll also show how you can load the data set as a CSV onto AWS S3.

## Import setup packages

In [1]:
import pandas as pd
import tomli
import pprint

# We'll use random type functions to generate the random house price data
import random

## Import config using a TOML file

We'll store our config in a TOML file. The config will contain basic instructions for creating the test data. There are 4 tiers, each with different constraints. For example, in tier 1, we create house with prices in the range 100k to 200k.


_Contents of config/housing.toml_

```yaml
# Config for creating a housing dataset

[tier-1]
price.range = "100000-200000"
bedrooms.range = "1-3"
bathrooms.range = "1-3"
council_tax.list = ["A", "B", "C"]
freehold.list = [true, false]
garage.list = [false]
parking.list = [true, false]

[tier-2]
price.range = "200000-300000"
bedrooms.range = "2-4"
bathrooms.range = "2-4"
council_tax.list = ["B", "C", "D"]
freehold.list = [true, false]
garage.list = [true, false]
parking.list = [true, false]

[tier-3]
price.range = "300000-400000"
bedrooms.range = "3-5"
bathrooms.range = "3-5"
council_tax.list = ["D", "E", "F"]
freehold.list = [true]
garage.list = [true, false]
parking.list = [true, false]

[tier-4]
price.range = "400000-1000000"
bedrooms.range = "3-8"
bathrooms.range = "3-8"
council_tax.list = ["D", "E", "F"]
freehold.list = [true]
garage.list = [true, false]
parking.list = [true]

```

In [2]:
with open('config/housing.toml', 'rb') as file_obj:
    housing_config = tomli.load(file_obj)
    
pprint.pprint(housing_config["tier-1"])

{'bathrooms': {'range': '1-3'},
 'bedrooms': {'range': '1-3'},
 'council_tax': {'list': ['A', 'B', 'C']},
 'freehold': {'list': [True, False]},
 'garage': {'list': [False]},
 'parking': {'list': [True, False]},
 'price': {'range': '100000-200000'}}


## Build random data using the config file

Next we build our random DataFrame using the TOML file. We use two main functions (plus some helper functions):

1. The first function `generate_one_row_random_house_data` creates a random row of data per the config specification
2. The second main function `generate_housing_df` uses the first function to create a large sample of data

As always we try to keep our functions small:

> <i>"The first rule of functions is that they should be small. The second rule of functions is that they should be smaller than that."</i>  -Robert C. Martin, Clean Code: A Handbook of Agile Software Craftsmanship

In [3]:
def get_randint_from_range_str(range_str):
    
    """
    Randomly returns an int in range
    The range is parsed from string in the form '2-10'
    If the value is greater than 100000, round to 3 decimal places 
    """
    
    range_list = [int(i) for i in range_str.split("-")]
    
    min_value, max_value = min(range_list), max(range_list)
    
    rand_int = random.randint(min_value, max_value)
    
    # For bigger numbers, such as price, we round to closest thousand
    # House prices are more likely to be quoted rounded to the nearest thousand
    
    return rand_int if rand_int <= 100000 else round(rand_int, -3)

In [4]:
def get_random_tier_config(housing_config):
    
        """
        Returns tier config for one tier.
        The config set is selected randomly
        """
    
        tiers = list(housing_config.keys())

        random_tier = random.choice(tiers)

        tier_config = housing_config[random_tier]
    
        return tier_config

In [5]:
def generate_one_row_random_house_data(config):
    
    """
    Returns one row of data in dict format
    The data is randomly produced using constraints in the config file
    """

    new_row = {}

    for housing_attribute in config:
        
        range_values = config[housing_attribute].get("range")
        
        list_values = config[housing_attribute].get("list")

        if range_values:
            
            new_row[housing_attribute] = get_randint_from_range_str(range_values)

        if list_values:

            new_row[housing_attribute] = random.choice(list_values)
            
    return new_row

In [6]:
def generate_housing_df(housing_config, row_size=1000000):
    
    """
    Returns a dataframe of random housing data per 
    constraints outlined in TOML file.
    The number of rows is specified in the parameters.
    """

    df_rows = []

    for i in range(row_size):

        tier_config = get_random_tier_config(housing_config)

        new_row = generate_one_row_random_house_data(tier_config)

        df_rows.append(new_row)

    return pd.DataFrame(df_rows)

In [7]:
df = generate_housing_df(housing_config)
df.head(3)

,price,bedrooms,bathrooms,council_tax,freehold,garage,parking
0,295000,3,4,B,True,True,False
1,151000,3,2,B,True,False,False
2,378000,5,3,D,True,True,False


In [8]:
# Check the statistical dispersion of the sample data

df.describe()

,price,bedrooms,bathrooms
count,1000000.000000,1000000.000000,1000000.000000
mean,362809.449000,3.626143,3.626549
std,226175.701698,1.703533,1.704032
min,100000.000000,1.000000,1.000000
25%,200000.000000,3.000000,3.000000
50%,300000.000000,3.000000,3.000000
75%,401000.000000,5.000000,5.000000
max,1000000.000000,8.000000,8.000000


## Save CSV file to S3

Finally, we save the CSV file to S3. Fortunately, the Pandas API does all of the "heavy lifting" to connect to AWS.

In [9]:
%%script echo skipping

S3_BUCKET_LOCATION = "<BUCKET_LOCATION>"

df.to_csv(S3_BUCKET_LOCATION)

skipping
